# About: HTTPS接続に変更する
---
Moodleコンテナへの接続を HTTP から HTTPS に変更します。

## 概要

「021-Moodleコンテナの起動.ipynb」で構築したMoodle環境ではHTTP(80)で接続する設定となっています。この設定を変更しHTTPS(443)でMoodleコンテナに接続できるように変更する手順を示します。

![処理の流れ](images/moodle-051-01.png)

主な手順は以下のようになります。

1. Moodleコンテナに `mod_ssl` パッケージを追加する
1. HTTPS接続を行うためのサーバ証明書を配置する
1. HTTPS接続に行うためのApache HTTP Serverの設定ファイル `ssl.conf` を編集する
1. `docker-compose.yml` を編集してMoodleの公開ポートなどを変更する

ApacheHTTP Serverに配置するサーバ証明書、秘密鍵ファイルは、このNotebook環境に事前にアップロードを行ってください。

## パラメータの指定

### グループ名の指定

このNotebookの操作対象となるAnsibleのグループ名を設定します。

In [ ]:
# (例)
# target_group = 'Moodle'

target_group =

#### チェック

指定された `target_group` の値が適切なものかチェックします。

`target_group` に対応する `group_vars` ファイルが存在していることを確認します。

In [ ]:
from pathlib import Path
if not (Path('group_vars') / (target_group + '.yml')).exists():
    raise RuntimeError(f"ERROR: not exists {target_group + '.yml'}")

`target_group`で指定したホストにAnsibleで到達可能であることを確認します。

In [ ]:
!ansible {target_group} -m ping

### 現在利用中のMoodleコンテナイメージの指定

HTTP接続の構成で利用しているMoodleコンテナイメージには`mod_ssl`パッケージがインストールされていません。そこで、このNotebookでは`mod_ssl`パッケージを追加した新たなMoodleコンテナのイメージを作成します。

操作対象のMoodle環境が「021-Moodleコンテナの起動.ipynb」によって作成された場合は `group_vars` ファイルにMoodleコンテナイメージ名が記録されているため、それを元に新たなコンテナイメージを作成することができます。しかし「821-AmazonS3からのリストア.ipynb」、「822-scpによるリストア.ipynb」によって復元された環境の場合、`group_vars` にMoodleコンテナのイメージ名が記録されていないため、ここで改めて指定する必要があります。

まず、`group_vars` にMoodleコンテナイメージが記録されているかどうかを確認します。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(target_group)
if 'moodle_image_name' not in gvars:
    raise RuntimeError()

上のセルの実行が正常終了となった場合はコンテナイメージ名が `group_vars` に記録されています。次章の「3. メンテナンスモードへの切り替え」までスキップしてください。

エラーになった場合は以降の手順で現在利用中のMoodleコンテナイメージを指定する必要があります。

現在実際に利用しているMoodleコンテナイメージを確認するために、構築環境の `docker-compose.yml` のコンテナイメージ指定の記述を確認します。

In [ ]:
!ansible {target_group} -m shell -a \
    'grep -w image /srv/moodle/docker-compose.yml | grep moodle'

上のセルの実行結果に表示された `harbor.vcloud.nii.ac.jp/vcp/moodle-simple:X.X.X` の部分がコンテナイメージ名になります。確認した値を次のセルに設定してください。

In [ ]:
# (例)
# moodle_image_name = 'harbor.vcloud.nii.ac.jp/vcp/moodle-simple:3.9.1'

moodle_image_name =

設定された値を `group_vars` に保存します。

In [ ]:
update_group_vars(target_group, moodle_image_name=moodle_image_name)

## メンテナンスモードへの切り替え

Moodleにアクセスするためのアドレスが変更される場合があるので、メンテナンスモードに切り替えます。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle docker-compose exec -T moodle \
    /usr/bin/php /var/www/html/admin/cli/maintenance.php --enable'

## Moodleコンテナイメージの作成

Moodleコンテナに `mod_ssl` パッケージを追加したイメージを作成します。

### コンテナイメージの作成

Moodleコンテナイメージに`mod_ssl` パッケージを追加したものを作成します。

コンテナイメージをビルドするための `Dockerfile` などを配置します。

In [ ]:
!ansible {target_group} -m file -a 'path=/srv/moodle/build state=directory'
!ansible {target_group} -m template -a 'dest=/srv/moodle/build/ src=template/Dockerfile'

配置した `Dockerfile` の内容を確認します。

In [ ]:
!ansible {target_group} -a 'cat /srv/moodle/build/Dockerfile'

コンテナイメージのビルドを行います。

In [ ]:
ssl_container_image = 'moodle-ssl'
!ansible {target_group} -a 'docker build -t {ssl_container_image} /srv/moodle/build/'

### 設定ファイルの配置

作成したコンテナイメージからHTTPS に関する設定ファイルを取り出し構築環境のホスト側に配置します。

この節で配置する設定ファイルを以下に示します。
<table>
  <tr>
    <th style="text-align:left;">コンテナ内のパス</th>
    <th style="text-align:left;">ホスト環境のパス</th>
  </tr>
  <tr>
    <td style="text-align:left;">/etc/httpd/conf.d/ssl.conf</td>
    <td style="text-align:left;">/srv/moodle/moodle/conf/httpd/conf.d/ssl.conf</td>
  </tr>
  <tr>
    <td style="text-align:left;">/etc/httpd/conf.modules.d/00-ssl.conf</td>
    <td style="text-align:left;">/srv/moodle/moodle/conf/httpd/conf.modules.d/00-ssl.conf</td>
  </tr>
</table>


設定ファイルを取り出すために一時的にコンテナを作成します。

In [ ]:
ssl_container_name = 'moodle-ssl'
!ansible {target_group} -a 'docker create --name {ssl_container_name} {ssl_container_image}'

設定ファイルをホスト側にコピーします。

In [ ]:
!ansible {target_group} -a \
    'docker cp {ssl_container_name}:/etc/httpd/conf.d/ssl.conf \
    /srv/moodle/moodle/conf/httpd/conf.d/'
!ansible {target_group} -a \
    'docker cp {ssl_container_name}:/etc/httpd/conf.modules.d/00-ssl.conf \
    /srv/moodle/moodle/conf/httpd/conf.modules.d/'

設定ファイルが配置されたことを確認します。

In [ ]:
!ansible {target_group} -a 'tree /srv/moodle/moodle/conf/httpd/'

設定ファイルをコピーするために一時的に作成したコンテナを削除します。

In [ ]:
!ansible {target_group} -a 'docker rm {ssl_container_name}'

## サーバ証明書の配置

Moodleコンテナで使用するサーバ証明書を配置します。

ここで配置するファイルを以下に示します。
<table>
  <tr>
    <th style="text-align:left;">コンテナ内のパス</th>
    <th style="text-align:left;">ホスト環境のパス</th>
    <th style="text-align:left;">対応する設定項目</th>
  </tr>
  <tr>
    <td style="text-align:left;">/etc/httpd/certs/server.crt</td>
    <td style="text-align:left;">/srv/moodle/moodle/conf/httpd/certs/server.crt</td>
    <td style="text-align:left;">SSLCertificateFile</td>
  </tr>
  <tr>
    <td style="text-align:left;">/etc/httpd/certs/server.key</td>
    <td style="text-align:left;">/srv/moodle/moodle/conf/httpd/certs/server.key</td>
    <td style="text-align:left;">SSLCertificateKeyFile</td>
  </tr>
</table>


### サーバ証明書

サーバ証明書を配置します。

このNotebook環境にアップロードしたサーバ証明書のパスを次のセルで指定してください。証明書はPEM形式であることを想定しています。

In [ ]:
# (例)
# moodle_cert_path = './certs/server.crt'

moodle_cert_path =

サーバ証明書をMoodle環境に配置します。

In [ ]:
import os

# 指定したパスに証明書ファイルが存在していることをチェックする
if not os.path.exists(moodle_cert_path):
    raise RuntimeError()

!ansible {target_group} -m file -a \
    'path=/srv/moodle/moodle/conf/httpd/certs state=directory'
!ansible {target_group} -b -m copy -a \
    'dest=/srv/moodle/moodle/conf/httpd/certs/server.crt \
    src={moodle_cert_path} owner=48 group=48'

サーバ証明書がMoodle環境に配置されたことを確認するために、証明書の内容を表示してみます。

In [ ]:
!ansible {target_group} -a \
    'openssl x509 -in /srv/moodle/moodle/conf/httpd/certs/server.crt \
    -noout -text'

### 秘密鍵の配置

このNotebook環境にアップロードしたサーバ証明書の秘密鍵ファイルのパスを次のセルで指定してください。

> ここで配置する秘密鍵ファイルにはパスフレーズの設定を行わないでください。

In [ ]:
# (例)
# moodle_private_key_path = './certs/server.key'

moodle_private_key_path =

秘密鍵を配置します。

In [ ]:
!ansible {target_group} -b -m copy -a \
    'dest=/srv/moodle/moodle/conf/httpd/certs/server.key \
    src={moodle_private_key_path} owner=48 group=48 mode=0400'

秘密鍵が配置されたことを確認するために、証明書ディレクトリのファイル一覧を表示してみます。

In [ ]:
!ansible {target_group} -a 'ls -l /srv/moodle/moodle/conf/httpd/certs/'

配置した秘密鍵のフォーマットが正しいことを確認します。

In [ ]:
!ansible {target_group} -b -a \
    'openssl rsa -in /srv/moodle/moodle/conf/httpd/certs/server.key \
    -noout -text'

## 設定ファイル ssl.conf の編集

設定ファイルを編集し Apache HTTP ServerのHTTPS接続に関する設定を行います。

設定ファイル`ssl.conf`を編集するために、構築環境からローカル環境にファイルを取得します。

In [ ]:
%run scripts/edit_conf.py
fetch_conf(target_group, 'moodle', '/etc/httpd/conf.d/ssl.conf')

上のセルの出力に表示されているリンクをクリックして設定ファイルの編集を行ってください。

HTTPS接続を有効にするには以下の設定を変更してください。

* ServerName
    - ホスト名を設定する
    - サーバ証明書と一致するホスト名を指定してください
* SSLCertificateFile
    - サーバ証明書のパス
    - 前章で配置したパス `/etc/httpd/certs/server.crt` を指定してください
* SSLCertificateKeyFile
    - サーバ証明書の秘密鍵のパス
    - 前章で配置したパス `/etc/httpd/certs/server.key` を指定してください

また必要に応じて `SSLProtocol`, `SSLCipherSuite` などの値も変更してください。

> メニューの[Edit]-[Find]で検索文字を入力することで、編集箇所に移動できます。
ファイルの編集後は**必ず**、メニューの[File]-[Save]を選択してファイルの保存を行ってください。

編集後に次のセルを実行すると編集前後の差分を確認することができます。

In [ ]:
show_local_conf_diff(target_group, 'moodle', '/etc/httpd/conf.d/ssl.conf')

表示される差分の例を以下に示します。

```
--- ssl.conf.orig
+++ ssl.conf
@@ -57,7 +57,7 @@
 
 #General setup for the virtual host, inherited from global configuration
 #DocumentRoot "/var/www/html"
-#ServerName www.example.com:443
+ServerName moodle.example.org:443
 
 #Use separate log files for the SSL virtual host; note that LogLevel
 #is not inherited from httpd.conf.
@@ -97,14 +97,14 @@
 #the certificate is encrypted, then you will be prompted for a
 #pass phrase.  Note that a kill -HUP will prompt again.  A new
 #certificate can be generated using the genkey(1) command.
-SSLCertificateFile /etc/pki/tls/certs/localhost.crt
+SSLCertificateFile /etc/httpd/certs/server.crt
 
 #Server Private Key:
 #If the key is not combined with the certificate, use this
 #directive to point at the key file.  Keep in mind that if
 #you've both a RSA and a DSA private key you can configure
 #both in parallel (to also allow the use of DSA ciphers, etc.)
-SSLCertificateKeyFile /etc/pki/tls/private/localhost.key
+SSLCertificateKeyFile /etc/httpd/certs/server.key
 
 #Server Certificate Chain:
 #Point SSLCertificateChainFile at a file containing the
```

編集したファイルを構築環境に配置します。

In [ ]:
apply_conf(target_group, 'moodle', '/etc/httpd/conf.d/ssl.conf', restart=False)

まだコンテナ設定の変更を行っていないため、ここでは設定ファイル反映のためのコンテナ再起動は行いません。

## コンテナ設定の変更

`docker-compose.yml` を編集してコンテナ設定の変更を行います。

### docker-compose.yml の編集

MoodleコンテナでHTTPS接続を有効にするには以下の設定変更が必要となります。

1. コンテナイメージの変更
   - mod_sslパッケージを追加したイメージに変更する
1. 公開ポートの変更
   - 公開ポート番号を 80 から 443 に変更する
1. 証明書ディレクトリの bind mount設定を追加
   - サーバ証明書をコンテナから参照できるようにするために bind mount の設定を追加します。
1. `MOODLE_URL`の変更
   - `config.php` の `$CFG->wwwroot` に設定するURLを https に変更する

コンテナ設定を変更するために `docker-compose.yml` をローカル環境に取得します。

In [ ]:
%run scripts/edit_conf.py
fetch_docker_compose(target_group)

上のセルの出力に表示されているリンクをクリックして設定ファイルの編集を行ってください。

Moodleコンテナのイメージを変更するには`docker-compose.yml` の `moodle` サービスに`build`の指定を追加してください。ビルドコンテキストとして指定するパスの値は `./build` となります。またコンテナイメージの名前も識別しやすい値に変更したほうが望ましいです。

変更後の例を以下に示します。

```
  moodle:
    image: moodle-ssl
    build: ./build
```

`docker-compose.yml` における `build` の指定方法の詳細については [Compose file version 3 reference - build](https://docs.docker.com/compose/compose-file/#build)を参照してください。

公開ポートの番号を変更するには `docker-compose.yml` の `ports` に指定している値を変更してください。デフォルトではポート番号の 80 を設定するため `80:80` を指定しています。これを `443:443` に変更してください。

変更後の例を以下に示します。

```
  moodle:
(中略)
    ports:
      - 443:443
```

`docker-compose.yml` における `ports` の指定方法の詳細については [Compose file version 3 reference - ports](https://docs.docker.com/compose/compose-file/#ports)を参照してください。

証明書を配置したディレクトリを bind mount に追加するには `volumes` の指定に証明書ディレクトリのエントリを追加します。
証明書を配置するディレクトリとしてホスト側は `/srv/moodle/moodle/conf/httpd/certs`、コンテナ側は `/etc/httpd/certs` を想定しています。そのため追加するエントリの値は`/srv/moodle/moodle/conf/httpd/certs:/etc/httpd/certs`となります。

変更後の例を以下に示します。

```
  moodle:
(中略)
    volumes:
      - /srv/moodle/moodle/conf/httpd/certs:/etc/httpd/certs
(以下略)
```

`docker-compose.yml` における `volumes` の指定方法の詳細については [Compose file version 3 reference - volumes](https://docs.docker.com/compose/compose-file/#volumes)を参照してください。

Moodleコンテナでは `config.php` の `$cfg->wwwroot` に設定する値をコンテナの環境変数 `MOODLE_URL` で外部から指定できるようにしています。HTTPS接続に変更するにあたって MoodleのURLを変更する場合は `docker-compose.yml` で指定している `MOODLE_URL` の値を変更してください。


変更後の例を以下に示します。
```
     environment:
       MOODLE_URL: https://moodle.example.org
```

編集後に次のセルを実行すると編集前後の差分を確認することができます。

In [ ]:
show_local_docker_compose_diff(target_group)

表示される差分の例を以下に示します。

```
--- docker-compose.yml.orig
+++ docker-compose.yml
@@ -20,7 +20,8 @@
       - --explicit_defaults_for_timestamp=true
       - --log-error=/var/log/mysql/error.log
   moodle:
-    image: harbor.vcloud.nii.ac.jp/vcp/moodle-simple:3.9.1
+    image: moodle-ssl
+    build: ./build
     container_name: moodle
     restart: always
     volumes:
@@ -28,16 +29,17 @@
       - /srv/moodle/moodle/data/php:/var/www/moodle:rw
       - /srv/moodle/moodle/conf/php.ini:/etc/php.ini
       - /srv/moodle/moodle/conf/php.d:/etc/php.d
+      - /srv/moodle/moodle/conf/httpd/certs:/etc/httpd/certs
       - /srv/moodle/moodle/conf/httpd/conf:/etc/httpd/conf
       - /srv/moodle/moodle/conf/httpd/conf.d:/etc/httpd/conf.d
       - /srv/moodle/moodle/conf/httpd/conf.modules.d:/etc/httpd/conf.modules.d
       - /srv/moodle/moodle/conf/ca-trust:/etc/pki/ca-trust/source/anchors
       - /var/log/httpd:/var/log/httpd
     ports:
-      - 80:80
+      - 443:443
     environment:
       TZ: Asia/Tokyo
-      MOODLE_URL: http://moodle.example.org
+      MOODLE_URL: https://moodle.example.org
       MOODLE_DATAROOT: /var/data/moodledata
       MOODLE_ADMIN: admin
       MOODLE_ADMIN_PASSWORD: "moodle"
```

### コンテナ設定の変更を反映する

`docker-compose.yml` を構築環境に配置して、コンテナ設定の変更反映をするためにコンテナを再起動します。

In [ ]:
apply_docker_compose(target_group)

変更後のコンテナの状態を確認してみます。`State` の値が `Up` となっていることを確認してください。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    docker-compose ps'

`State`が`Up`となっていない場合はコンテナのログなどを確認してください。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    docker-compose logs --tail=50 moodle'

## メンテナンスモードの解除

設定変更が完了したのでメンテナンスモードを解除します。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle docker-compose exec -T moodle \
    /usr/bin/php /var/www/html/admin/cli/maintenance.php --disable'

## Moodle を利用できることを確認

Moodle環境にアクセスして利用可能であることを確認します。

次のセルを実行すると`docker-compose.yml` に設定したMoodleのアドレスを表示します。表示されたリンクをクリックしてMoodleにアクセスできることを確認してください。

In [ ]:
local_path = get_local_host_conf(target_group, '/srv/moodle/docker-compose.yml')
!grep MOODLE_URL {local_path}